In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 12

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((279312, 12), 0)

### Unobserved Nodes = 10%

In [10]:
data = load_data(path_10)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 144000
Node 0
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.004 )
( 11.09762919988002 7.825636977922009 0.25266995489672706 0.6572049390540399 5.907211780548096
Node 1
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.033 )
( 13.608090470027502 10.101243139987213 0.32168827645829207 0.5814480386312941 6.035867929458618
Node 2
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.098 )
( 12.055628969634993 9.327033468495763 0.33621863122193435 0.6303553299452874 6.224366188049316
Node 3
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.74223859611654 31.02186594202898 1.0 0.0 0.01795196533203125
Node 4
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 16.548692604080756 12.895686780867218 0.37811738836569336 0.4955803732864206 7.121963739395142
Node 5
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.018 )
( 13.5909650879623 9.898567022596275 0.2860004167501784 0.5859203771393189 8.135258436203003
Node 6
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.89035059471634 31.137

( 13.26030628518487 10.65717161220869 0.4209322146655059 0.5999313805846873 6.088725805282593
Node 57
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 11.561360977996305 8.506769810867048 0.295413714314424 0.6503130659340111 6.230348825454712
Node 58
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.037 )
( 15.674469769449518 13.294903549769595 0.5326900889133145 0.524421371748812 5.345757961273193
Node 59
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 11.855580567987035 9.263797993531979 0.3558973505806864 0.6399388402366483 6.125586271286011
Node 60
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.053 )
( 11.311305099751626 8.82654738723784 0.3442335465380832 0.6555380844101879 5.859338998794556
Node 61
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.063 )
( 15.582015167856763 12.181983139951804 0.3545372856638878 0.5237705735833031 5.4793548583984375
Node 62
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.002 )
( 11.070986294699825 8.448937341345607 0.3207838023145972 0.6606528

( 10.128107448771225 7.830640301635332 0.3875799329031088 0.6222098341709825 6.686589479446411
Node 111
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 9.683441264145099 7.008802477682323 0.3108661142760238 0.6386992773411043 6.12662672996521
Node 112
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.027 )
( 9.877403714875497 7.445535666974382 0.36272958192542987 0.6299924047941312 6.261796951293945
Node 113
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 11.083685673726256 8.45277607018461 0.38115269583299166 0.5842516192649901 5.758610248565674
Node 114
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.042 )
( 9.23011986966611 6.762994906052609 0.34236832122042066 0.6525677675808883 7.4361279010772705
Node 115
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 10.30269189122007 7.71218656705636 0.3721299302700862 0.6122794236805384 6.928482294082642
Node 116
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.05 )
( 11.85633810072929 9.770826724946625 0.5347546587045311 0.5521303613

( 11.707444918235145 9.291990469547315 0.47986394149558964 0.5540952724779387 5.314146518707275
Node 166
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.064 )
( 17.465537930915527 14.47979920122853 0.7511744029474364 0.3364880027719882 4.290635108947754
Node 167
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.061 )
( 17.816209794715263 14.466867991687621 0.8035931143863815 0.3185346883742348 4.39321494102478
Node 168
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.003 )
( 29.23042204251607 27.541538502018955 1.5008959956837342 -0.11642293772116408 6.156545639038086
Node 169
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.009 )
( 16.087285822357437 13.222837049850755 0.657105484122362 0.38590820316727825 3.8955886363983154
Node 170
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.024 )
( 10.225660716178625 7.5361948224035205 0.35698043044901967 0.6083854996739473 5.756616592407227
Node 171
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.006 )
( 12.772032055461478 10.508934187936953 0.57517484905139

( 12.270534748048272 9.933281138377554 0.4469515683228395 0.5791416412220718 6.252579689025879
Node 220
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.006 )
( 12.750633556252817 10.202945378943989 0.4557573444453542 0.563343610720473 5.7745654582977295
Node 221
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.017 )
( 10.453664555958817 8.119568917853519 0.356204605073299 0.6416320689791588 6.227629899978638
Node 222
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.015 )
( 10.138553762899706 7.118024976668019 0.2725625148892024 0.6524293434111819 6.8222808837890625
Node 223
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.024 )
( 10.758812316288681 7.933088070613977 0.316067807999029 0.6319839386799477 6.823762655258179
Node 224
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 29.185961550037057 27.508626207729474 1.0 0.0 0.014959573745727539
Node 225
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.03 )
( 10.341942805238483 7.6267133781555865 0.3122886018287328 0.6450685896080861 6.6808521747

( 13.7238320313894 10.07967646143745 0.2982446122913329 0.5963519220560718 5.959075689315796
Node 274
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.002 )
( 16.6165094578717 12.930671944588646 0.366954506564975 0.5110161490196022 5.59909725189209
Node 275
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.023 )
( 15.927068724607153 12.21736760814255 0.34526784473944083 0.5321763432873339 5.875805616378784
Node 276
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 34.137076754089726 32.27238073671497 1.0 0.0 0.0169522762298584
Node 277
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.132 )
( 18.057324927871694 14.547090811201548 0.4108738317421206 0.4708771164433556 6.299178600311279
Node 278
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.052 )
( 14.714376748841909 11.0149080206002 0.315969392352481 0.5699194934770717 7.069108009338379
Node 279
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.044 )
( 15.823031397451732 12.120624656769948 0.3454341011214344 0.5382275180468405 6.379184246063232
Nod

( 16.721131835615726 14.047134265748225 0.5819609549502786 0.49355215373818284 6.8107688426971436
Node 328
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.90756398170708 31.08936594202898 1.0 0.0 0.01196742057800293
Node 329
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.048 )
( 12.321059787807302 9.325568622892225 0.336529099003915 0.6247148546825665 6.402717590332031
Node 330
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.79681982464006 30.99267210144927 1.0 0.0 0.012020111083984375
Node 331
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.008 )
( 12.37520141869566 9.872772035982756 0.40551462151055845 0.6227895558852881 5.104307413101196
Node 332
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.026 )
( 12.159007406990549 9.284057877237236 0.3480502207140858 0.6272855551786976 5.373666763305664
Node 333
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.099 )
( 13.308982279278583 11.118452632998753 0.4671961746249547 0.5907416498527931 5.7266669273376465
Node 334
( (2280, 12) (228

( 11.150288950588193 8.47687554551126 0.40140893848955833 0.5787355851307019 5.810473442077637
Node 382
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.035 )
( 13.10514401104115 10.182214221985552 0.4693278669364043 0.5038816551747091 5.458410263061523
Node 383
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.034 )
( 10.223506438201161 7.721618325739583 0.3967294755369574 0.611336172771713 6.109452247619629
Node 384
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.056 )
( 10.426039647993255 7.717636337118655 0.37714599850088054 0.602727313783181 5.439033269882202
Node 385
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 26.1980089872174 24.31232638888889 1.0 0.0 0.011928558349609375
Node 386
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 10.785622576815298 8.580710809827957 0.46489675309749745 0.5885390013228657 6.763838529586792
Node 387
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.031 )
( 12.240853980824738 9.894816913538605 0.5318151433421591 0.5319857899815204 5.172770738601685

( 10.290774481193631 7.421124358140708 0.38018053571106086 0.5947805333748567 5.204092502593994
Node 435
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.07 )
( 10.224664599883571 7.368545363760126 0.3658024216378759 0.5965280495494956 5.3271636962890625
Node 436
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.013 )
( 11.311902779454666 8.0853513150421 0.34523086905624895 0.5527042285627353 4.840079307556152
Node 437
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 10.823381045776092 8.103661042381495 0.43708602057149637 0.5721046984069691 4.410213232040405
Node 438
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.019 )
( 10.281618586629083 7.93443668260224 0.4649802829320002 0.5923588627714207 6.235367059707642
Node 439
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.023 )
( 11.312097382603772 9.253760955684232 0.5530058426983607 0.5527597163110447 6.0039849281311035
Node 440
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.049 )
( 14.90183747090121 12.872275419446526 0.7631926049033897 0.

( 14.37103429450857 12.66906679341542 0.5739599798397973 0.5136736309422834 5.602037191390991
Node 488
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.016 )
( 10.525738876093476 8.264460833007563 0.35630866727836813 0.6425567633874707 6.058809757232666
Node 489
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.029 )
( 17.2845272629988 15.620849008822674 0.709233699961325 0.41348894064056674 5.01060676574707
Node 490
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.027 )
( 14.843180832924745 13.059571870316773 0.6020007565971067 0.49528162053858193 6.758945465087891
Node 491
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.026 )
( 10.44136128617237 7.483606188998042 0.2826400204261379 0.644296210959719 5.086366891860962
Node 492
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.013 )
( 12.513622292260733 9.155882997352007 0.31343026589121165 0.5746286002882366 5.871308088302612
Node 493
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 29.494979421093426 27.867569444444452 1.0 0.0 0.01296401023864

### Unobserved Nodes = 20%

In [11]:
data = load_data(path_20)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 290880
Node 0
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.004 )
( 11.09762919988002 7.825636977922009 0.25266995489672706 0.6572049390540399 5.426527500152588
Node 1
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.033 )
( 13.608090470027502 10.101243139987213 0.32168827645829207 0.5814480386312941 4.638571739196777
Node 2
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.098 )
( 12.055628969634993 9.327033468495763 0.33621863122193435 0.6303553299452874 4.866025447845459
Node 3
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.74223859611654 31.02186594202898 1.0 0.0 0.010939836502075195
Node 4
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 16.548692604080756 12.895686780867218 0.37811738836569336 0.4955803732864206 5.227030038833618
Node 5
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.018 )
( 13.5909650879623 9.898567022596275 0.2860004167501784 0.5859203771393189 6.010967016220093
Node 6
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.013 )
( 11.969460367350356 8.

( 10.736110037698607 7.735044537158419 0.2669174087690284 0.6753759804716724 6.358007907867432
Node 56
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.003 )
( 13.26030628518487 10.65717161220869 0.4209322146655059 0.5999313805846873 5.032585620880127
Node 57
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 11.561360977996305 8.506769810867048 0.295413714314424 0.6503130659340111 4.8001368045806885
Node 58
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.037 )
( 15.674469769449518 13.294903549769595 0.5326900889133145 0.524421371748812 4.434184312820435
Node 59
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 11.855580567987035 9.263797993531979 0.3558973505806864 0.6399388402366483 5.101361513137817
Node 60
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.83760725879844 31.13514190821257 1.0 0.0 0.011936426162719727
Node 61
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.063 )
( 15.582015167856763 12.181983139951804 0.3545372856638878 0.5237705735833031 4.311506032943726
Nod

( 10.128107448771225 7.830640301635332 0.3875799329031088 0.6222098341709825 5.792550086975098
Node 111
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 9.683441264145099 7.008802477682323 0.3108661142760238 0.6386992773411043 5.265894651412964
Node 112
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 26.69513772921283 25.151820652173924 1.0 0.0 0.01196742057800293
Node 113
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 11.083685673726256 8.45277607018461 0.38115269583299166 0.5842516192649901 4.862006425857544
Node 114
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.042 )
( 9.23011986966611 6.762994906052609 0.34236832122042066 0.6525677675808883 6.100725173950195
Node 115
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 10.30269189122007 7.71218656705636 0.3721299302700862 0.6122794236805384 5.9720458984375
Node 116
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.05 )
( 11.85633810072929 9.770826724946625 0.5347546587045311 0.5521303613228427 5.0764665603637695
Node 1

( 17.465537930915527 14.47979920122853 0.7511744029474364 0.3364880027719882 3.706089735031128
Node 167
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.061 )
( 17.816209794715263 14.466867991687621 0.8035931143863815 0.3185346883742348 3.782892942428589
Node 168
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.003 )
( 29.23042204251607 27.541538502018955 1.5008959956837342 -0.11642293772116408 5.268916845321655
Node 169
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.009 )
( 16.087285822357437 13.222837049850755 0.657105484122362 0.38590820316727825 3.420853853225708
Node 170
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.024 )
( 10.225660716178625 7.5361948224035205 0.35698043044901967 0.6083854996739473 4.859020948410034
Node 171
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.006 )
( 12.772032055461478 10.508934187936953 0.5751748490513929 0.5094638810831712 4.948768377304077
Node 172
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.019 )
( 13.597047364127612 10.81269565480712 0.492183666683242

( 10.453664555958817 8.119568917853519 0.356204605073299 0.6416320689791588 5.046483755111694
Node 222
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.015 )
( 10.138553762899706 7.118024976668019 0.2725625148892024 0.6524293434111819 5.1991331577301025
Node 223
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.024 )
( 10.758812316288681 7.933088070613977 0.316067807999029 0.6319839386799477 5.317758560180664
Node 224
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 29.185961550037057 27.508626207729474 1.0 0.0 0.01496124267578125
Node 225
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 29.137862985467958 27.469519927536236 1.0 0.0 0.012964248657226562
Node 226
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.01 )
( 11.010747034428627 7.9169486740896176 0.29249645858556267 0.6216470151264311 5.36568808555603
Node 227
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 29.092466020718604 27.44252415458938 1.0 0.0 0.013968944549560547
Node 228
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.

( 15.823031397451732 12.120624656769948 0.3454341011214344 0.5382275180468405 5.478388547897339
Node 280
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 13.229224907658557 9.698911159643798 0.29964151862187166 0.6148946276307508 5.787533760070801
Node 281
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.052 )
( 15.699971232352304 11.950442551277924 0.34400239590566684 0.544913610310712 5.8643903732299805
Node 282
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 15.768386698332655 12.092354347546378 0.37713815938988726 0.5446197987541219 4.95775032043457
Node 283
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.008 )
( 14.193783046864304 10.69540389835194 0.3344246890190103 0.5914634024852159 4.362342357635498
Node 284
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.018 )
( 11.733963311502087 8.672395866031316 0.3139399188933725 0.6617713406143165 5.172175168991089
Node 285
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.005 )
( 17.99779991547447 14.311972094737934 0.4095375655280053 0.

( 19.98473654162992 18.153504078619907 0.7568190411035466 0.38291331100480186 5.62297248840332
Node 335
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 32.23956025062975 30.539829408212558 1.0 0.0 0.013960599899291992
Node 336
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.001 )
( 12.338550361152736 10.078269835165992 0.42353300103420527 0.6155502033093219 5.899235010147095
Node 337
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.034 )
( 10.667482135294689 7.80394613754929 0.291451877762858 0.667252065727294 5.519247770309448
Node 338
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.041 )
( 17.68369166576138 15.803157521992167 0.6764206211096256 0.4470029834809264 4.639601469039917
Node 339
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.036 )
( 11.221242097022847 8.644079507565046 0.34956759436030826 0.6475894735763399 5.063467502593994
Node 340
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.025 )
( 10.875160558954262 8.223622655830653 0.32321137563480024 0.6565281363950279 5.151233196258

( 23.048823907327566 21.306210501666616 1.198383785096823 0.11399707544817606 4.256622552871704
Node 390
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 25.883653334948068 23.880845410628012 1.0 0.0 0.010970830917358398
Node 391
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.017 )
( 10.95673047061927 7.711745605060419 0.3408271646384861 0.577073482486284 5.122308969497681
Node 392
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.005 )
( 10.680713367872988 7.583576224266815 0.3550871668324888 0.5866923100213858 5.736669540405273
Node 393
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.023 )
( 11.840888767123575 9.05113743098611 0.47504445760861985 0.5401125904917117 4.692458868026733
Node 394
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.015 )
( 10.453640935371132 7.855256980431697 0.43872983363423784 0.5919967399872732 4.5119407176971436
Node 395
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.033 )
( 11.530638693631413 8.504521408341411 0.40886082461786527 0.5483327973046561 5.41851735

( 10.217971973968906 7.35428848259442 0.3631476362775549 0.596205432189658 4.735344886779785
Node 444
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 25.263801073890036 23.341858393719807 1.0 0.0 0.01296377182006836
Node 445
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.037 )
( 13.713508048025988 10.440439156775069 0.44039930900259566 0.4584558917837499 3.6891419887542725
Node 446
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 11.168884573942227 8.796750326136186 0.4979064119460234 0.5588617252800125 4.514932870864868
Node 447
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.026 )
( 10.099304669219116 7.784173408820735 0.43815401980802865 0.6015247996782257 4.6435887813568115
Node 448
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 25.40917138741063 23.496843297101435 1.0 0.0 0.010971307754516602
Node 449
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.054 )
( 10.691702927612203 7.857022855855945 0.3814331940011295 0.5810042481609476 5.974067687988281
Node 450
( (2280, 12) (2

( 10.9925142472907 8.791353490584905 0.3785598293001247 0.6328783891691322 5.307814359664917
Node 499
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.004 )
( 10.038137247816001 7.337215240520894 0.28667272196430643 0.6668185674752926 4.210745096206665
Node 500
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.012 )
( 10.312903935304066 7.867116376660145 0.31854173315095496 0.6597551408900486 4.183818817138672
Node 501
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 10.175943470411775 7.410033986384267 0.29010775821608925 0.6662940500497725 4.709414482116699
Node 502
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.01 )
( 11.336004071328505 8.393989160399716 0.3155375237252654 0.630858671565542 4.094056129455566
Node 503
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 30.937756213744123 29.247288647342987 1.0 0.0 0.011969327926635742
Node 504
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 31.077057762240123 29.384717693236706 1.0 0.0 0.011968135833740234
Node 505
( (2280, 12) 